In [1]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm

In [2]:
tradingPairs = {'JBF':'3443','QWF':'2388','HCF':'2498','DBF':'2610','EHF':'1319','IPF':'3035','IIF':'3006','QXF':'2615','PEF':'5425','NAF':'3105'}

In [3]:
stockPath = "../tickData/stocks"
stockFiles = os.listdir(stockPath)
futuresPath = "../tickData/futuresOB"
futuresFiles = os.listdir(futuresPath)
stockFiles.remove('.DS_Store')
futuresFiles.remove('.DS_Store')

In [4]:
stockData = {}
futuresData = {}
for file in stockFiles:
  stockData[file] = os.listdir(stockPath + '/' + file)
for file in futuresFiles:
  futuresData[file] = os.listdir(futuresPath + '/' + file)

In [5]:
stockTick = pd.read_csv('/Users/jeremyg/Documents/GitHub/SummerIndependentProject/tickData/stocks/1319/1319_md_202007_202007.csv.gz', compression='gzip')
stockTick

,Unnamed: 0,date,time,lastPx,size,volume,SP5,SP4,SP3,SP2,...,SV5,SV4,SV3,SV2,SV1,BV1,BV2,BV3,BV4,BV5
0,0,2020-07-01,90023105,3670.0,5.0,5,3700.0,3695.0,3690.0,3680.0,...,14,3,1,1,8,1,7,20,21,3
1,1,2020-07-01,90025201,NaN,NaN,5,3700.0,3695.0,3690.0,3680.0,...,14,4,1,1,8,1,7,20,21,3
2,2,2020-07-01,90035492,3670.0,1.0,6,3700.0,3695.0,3690.0,3680.0,...,14,4,1,1,7,1,7,20,21,3
3,3,2020-07-01,90039927,NaN,NaN,6,3700.0,3695.0,3690.0,3680.0,...,14,4,1,1,7,1,9,20,21,3
4,4,2020-07-01,90053917,3670.0,5.0,11,3700.0,3695.0,3690.0,3680.0,...,14,4,1,1,2,1,9,20,21,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43440,960,2020-07-31,132431079,3450.0,3.0,290,3480.0,3475.0,3470.0,3465.0,...,20,4,44,5,24,13,4,7,6,24
43441,961,2020-07-31,132431141,NaN,NaN,290,3480.0,3475.0,3470.0,3465.0,...,18,4,44,5,24,13,4,7,6,24
43442,962,2020-07-31,132432238,NaN,NaN,290,3480.0,3475.0,3470.0,3465.0,...,18,4,44,5,24,3,13,4,7,6
43443,963,2020-07-31,132441213,3455.0,1.0,291,3480.0,3475.0,3470.0,3465.0,...,18,4,44,5,24,2,13,4,7,6


In [6]:
stockPriceFiveMinute = {}
futuresPriceFiveMinute = {}

In [7]:
for stock in tqdm(stockData.keys()):
  stockPriceFiveMinute[stock] = pd.DataFrame()
  for date in stockData[stock]:
    stockTick = pd.read_csv(stockPath + '/' + stock + '/' + date, compression='gzip')
    #Data processing
    stockTick = stockTick.loc[stockTick['SP1'] * stockTick['BP1'] != 0]
    stockTick['date'] = pd.to_datetime(stockTick['date'], format="%Y-%m-%d")
    # get the column 'SP1','BP1' and calculate the mid quote
    stockTick['midQuote'] = np.log(stockTick[['SP1','BP1']].mean(axis=1))
    # set a timestamp every 5 min starting from 9:05, if the time is smaller than the tim
    stockTick['new_time'] =  stockTick['time'] +(500000- stockTick['time'] % 500000)
    # if the last 7 digits in new_time == 6000000, then add 4000000 to new_time
    stockTick['new_time'] = stockTick['new_time'] + (stockTick['new_time'] % 10000000 == 6000000) * 4000000
    stockTick['TimeStamp'] = pd.to_datetime(stockTick.date.astype(str) + ' ' + stockTick.new_time.astype(str), format="%Y-%m-%d %H%M%S%f")
    stockTick['Stock_Ask_Bid_Price_Depth_Ratio'] = (stockTick['SP5']- stockTick['SP1'])/ (stockTick['BP1']- stockTick['BP5'])
    ask_vol = stockTick[['SV1','SV2','SV3','SV4','SV5']].sum(axis=1)
    bid_vol = stockTick[['BV1','BV2','BV3','BV4','BV5']].sum(axis=1)
    stockTick['Stock_Ask_Bid_Total_Volume_Ratio'] = ask_vol/bid_vol
    stockPrice = stockTick[['TimeStamp','midQuote',  'volume', 'Stock_Ask_Bid_Price_Depth_Ratio', 'Stock_Ask_Bid_Total_Volume_Ratio']]
    stockPrice_ = stockPrice.groupby(['TimeStamp']).mean()
    stockPriceFiveMinute[stock] = pd.concat([stockPriceFiveMinute[stock], stockPrice_], axis=0)
  stockPriceFiveMinute[stock] = stockPriceFiveMinute[stock].sort_index()

100%|██████████| 10/10 [11:24<00:00, 68.49s/it]


In [8]:
stockPriceFiveMinute

{'2610':                      midQuote        volume  Stock_Ask_Bid_Price_Depth_Ratio  \
 TimeStamp                                                                      
 2020-07-01 09:05:00  6.714164    133.897196                          1.00000   
 2020-07-01 09:10:00  6.714296    220.728261                          0.93913   
 2020-07-01 09:15:00  6.714030    302.317073                          1.00000   
 2020-07-01 09:20:00  6.713130    375.428571                          1.00000   
 2020-07-01 09:25:00  6.712315    459.378378                          1.00000   
 ...                       ...           ...                              ...   
 2023-06-30 13:05:00  7.875689  49147.506127                          1.00000   
 2023-06-30 13:10:00  7.875689  49947.885267                          1.00000   
 2023-06-30 13:15:00  7.873951  51607.447162                          1.00000   
 2023-06-30 13:20:00  7.873759  54236.161032                          1.00000   
 2023-06-30 13:25:00

In [9]:
for futures in tqdm(futuresData.keys()):
  futuresPriceFiveMinute[futures] = pd.DataFrame()
  for date in futuresData[futures]:
    futuresTick = pd.read_csv(futuresPath + '/' + futures + '/' + date)
    #Data processing
    futuresTick = futuresTick.loc[futuresTick['askPrice1'] * futuresTick['bidPrice1'] != 0]
    futuresTick['date'] = pd.to_datetime(futuresTick['date'], format="%Y-%m-%d")
    # get the column 'SP1','BP1' and calculate the mid quote
    futuresTick['midQuote'] = np.log(futuresTick[['askPrice1','bidPrice1']].mean(axis=1))
    # past 5 min average
    futuresTick['new_time'] =  futuresTick['time'] +(500000- futuresTick['time'] % 500000)
    # if the last 7 digits in new_time == 6000000, then add 4000000 to new_time
    futuresTick['new_time'] = futuresTick['new_time'] + (futuresTick['new_time'] % 10000000 == 6000000) * 4000000
    futuresTick['TimeStamp'] = pd.to_datetime(futuresTick.date.astype(str) + ' ' + futuresTick.new_time.astype(str), format="%Y-%m-%d %H%M%S%f")
    futuresTick['Futures_Ask_Bid_Price_Depth_Ratio'] = (futuresTick['askPrice5']- futuresTick['askPrice1'])/ (futuresTick['bidPrice1']- futuresTick['bidPrice5'])
    ask_vol = futuresTick[['askSize1','askSize2','askSize3','askSize4','askSize5']].sum(axis=1)
    bid_vol = futuresTick[['bidSize1','bidSize2','bidSize3','bidSize4','bidSize5']].sum(axis=1)
    futuresTick['Futures_Ask_Bid_Total_Volume_Ratio'] = ask_vol/bid_vol
    futuresPrice = futuresTick[['TimeStamp','midQuote', 'Futures_Ask_Bid_Price_Depth_Ratio', 'Futures_Ask_Bid_Total_Volume_Ratio']]
    futuresPrice_ = futuresPrice.groupby(['TimeStamp'])['midQuote'].mean()
    futuresPriceFiveMinute[futures] = pd.concat([futuresPriceFiveMinute[futures], futuresPrice_], axis=0)
  futuresPriceFiveMinute[futures] = futuresPriceFiveMinute[futures].sort_index()

100%|██████████| 10/10 [03:06<00:00, 18.66s/it]


In [10]:
futuresPriceFiveMinute

{'IPF':                              0
 2020-07-01 08:50:00   8.428568
 2020-07-01 08:55:00   8.440785
 2020-07-01 09:00:00   8.447843
 2020-07-01 09:05:00   8.450001
 2020-07-01 09:10:00   8.435341
 ...                        ...
 2022-03-31 13:25:00  10.319801
 2022-03-31 13:30:00  10.321673
 2022-03-31 13:35:00  10.323292
 2022-03-31 13:40:00  10.320911
 2022-03-31 13:45:00  10.321375
 
 [23682 rows x 1 columns],
 'HCF':                             0
 2020-07-01 08:50:00  8.118272
 2020-07-01 08:55:00  8.117322
 2020-07-01 09:00:00  8.119836
 2020-07-01 09:05:00  8.106474
 2020-07-01 09:10:00  8.098645
 ...                       ...
 2023-06-30 13:25:00  8.651734
 2023-06-30 13:30:00  8.651870
 2023-06-30 13:35:00  8.651870
 2023-06-30 13:40:00  8.651724
 2023-06-30 13:45:00  8.653558
 
 [43168 rows x 1 columns],
 'DBF':                             0
 2022-08-31 08:50:00  7.708260
 2022-08-31 08:55:00  7.706721
 2022-08-31 09:00:00  7.705261
 2022-08-31 09:05:00  7.705539
 2022-08-3

In [11]:
spreads = {}
for futures in futuresData.keys():
  spreads[futures] = (stockPriceFiveMinute[tradingPairs[futures]] - futuresPriceFiveMinute[futures]).dropna()
  stockPriceFiveMinute[tradingPairs[futures]].to_csv('../tickData/midQuoteData/stock/' +tradingPairs[futures] + '_mid_quote.csv')
  futuresPriceFiveMinute[futures].to_csv('../tickData/midQuoteData/futures/' + futures + '_mid_quote.csv')